In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

import time
from time import localtime as lt

from astropy.coordinates import SkyCoord
import seaborn as seaborn
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import Planck15 as cosmo

### 加载数据以及基本信息

In [3]:
data_ori_z_05_35=pd.read_csv('data_ori_z_00_35_sort_by_mpc.csv')     # 所有数据
data_background=pd.read_csv('data_used_z_le_4.csv')     # 背景数据

In [3]:
len(data_ori_z_05_35)

181320

In [ ]:
r_kpc=[20,40,60,80]
# r_kpc=[20,40,60,80,100,200,400,500,800,1000,1200,1400,2000,2200]


neighbour=-99*np.ones((len(data_ori_z_05_35),len(r_kpc)))
neighbour_kde_sum=-99*np.ones((len(data_ori_z_05_35),len(r_kpc)))
t0=time.time()
count=0

for kpc_index,lim in enumerate(r_kpc):

    lim=lim/1000    # mpc
    lim_=lim+1


    print(str(lim)+'mpc',end=' ')
    print('time:{}:{}:{} '.format(lt().tm_hour,lt().tm_min,lt().tm_sec),end=' ')
    kde_acc=[]

    for ra1,dec1,mpc1,i in zip(data_ori_z_05_35['RA'],data_ori_z_05_35['DEC'],data_ori_z_05_35['comoving_radial_distance_mpc'],range(len(data_ori_z_05_35))):
        c1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree,distance=mpc1*u.Mpc)
        frame_tem=data_background[(data_background['comoving_radial_distance_mpc']>mpc1-lim_)&(data_background['comoving_radial_distance_mpc']<mpc1+lim_)]

        for ra2,dec2,mpc2,kde_ in zip(frame_tem['RA'],frame_tem['DEC'],frame_tem['comoving_radial_distance_mpc'],frame_tem['kde_normalisation']):
            # if np.abs(mpc1-mpc2)>1800:
                # if mpc1+lim_>mpc2>mpc1-lim_:    # 优化速度

                    c2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree,distance=mpc2*u.Mpc)

                    if c1.separation_3d(c2).value <=lim:

                        kde_acc.append(kde_)
                        count+=1

        neighbour[i,kpc_index]=count-1
        neighbour_kde_sum[i,kpc_index]=sum(kde_acc)
        count=0
        kde_acc=[]

    print('spend:{:.2f} min'.format((time.time()-t0)/60))

neighbour_3D_seperation=pd.DataFrame(neighbour)
kde_sum=pd.DataFrame(neighbour_kde_sum)

0.02mpc time:0:17:52  

In [ ]:
exec("neighbour_3D_seperation.to_csv('count_neighbour_{}_{}.csv',index=False)".format(r_kpc[0],r_kpc[-1]))
exec("kde_sum.to_csv('count_kde_{}_{}.csv',index=False)".format(r_kpc[0],r_kpc[-1]))